# Storing data in a BIG txt file

Data sources:
- Mozilla Common Voice

In [ ]:
'''
%%capture
! pip install datasets
! huggingface-cli login
'''


In [ ]:
'''
from datasets import load_dataset

dataset = load_dataset("mozilla-foundation/common_voice_7_0", "it", use_auth_token=True, split="train")

with open("text.txt", "w") as f:
    f.write(" ".join(dataset["sentence"]))
'''

# Updated version - Multi dataset
Data sources:
- Mozilla Common Voice 7.0
- LibriSpeech
- Multilingual TED
- VoxForge
- M-AILABS Speech
- EuroParl-ST
- EMOVO
- MSPKA

In [1]:
! wget https://huggingface.co/dbdmg/robust-asr-italian/resolve/main/kenlm_training_text.txt

--2022-01-28 10:10:47--  https://huggingface.co/dbdmg/robust-asr-italian/resolve/main/kenlm_training_text.txt
Resolving huggingface.co (huggingface.co)... 54.205.126.93, 34.224.55.150, 52.22.206.115, ...
Connecting to huggingface.co (huggingface.co)|54.205.126.93|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/dbdmg/robust-asr-italian/515260ce59e3fd677bd279ad4a2fa3570bf7ffbbcf383ac7a8ce068dc3fdd771 [following]
--2022-01-28 10:10:47--  https://cdn-lfs.huggingface.co/dbdmg/robust-asr-italian/515260ce59e3fd677bd279ad4a2fa3570bf7ffbbcf383ac7a8ce068dc3fdd771
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.224.154.51, 13.224.154.37, 13.224.154.121, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|13.224.154.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32015545 (31M) [binary/octet-stream]
Saving to: ‘kenlm_training_text.txt’

kenlm_training_text 100%[=================

# Training and storing KenLM-it-5

In [2]:
! sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
! wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
! mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
! ls kenlm/build/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.65.1.0ubuntu1).
libboost-thread-dev set to manually installed.
liblzma-dev is already the newest version (5.2.2-1.3).
liblzma-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
Su

In [3]:
! kenlm/build/bin/lmplz -o 5 < "kenlm_training_text.txt" > "5gram-it-multi-ds.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm_training_text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1918697472 bytes == 0x563e4863e000 @  0x7f08e9c4d1e7 0x563e4645b7a2 0x563e463f651e 0x563e463d52eb 0x563e463c1066 0x7f08e7de6bf7 0x563e463c2baa
tcmalloc: large alloc 8953896960 bytes == 0x563ebac0e000 @  0x7f08e9c4d1e7 0x563e4645b7a2 0x563e4644a7ca 0x563e4644b208 0x563e463d5308 0x563e463c1066 0x7f08e7de6bf7 0x563e463c2baa
****************************************************************************************************
Unigram tokens 5215793 types 301849
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:3622188 2:1062841216 3:1992827392 4:3188523776 5:4649930752
tcmalloc: large alloc 4649934848 bytes == 0x563e4863e000 @  0x7f08e9c4d1e7 0x563e4645b7a2 0x563e4644a7ca 0x563e4644b208 0x563e463d58d7 0x563e463c1066 0x7f08e7de6bf7 0x563e463c2baa
tcmalloc: large alloc 199283

In [4]:
with open("5gram-it-multi-ds.arpa", "r") as read_file, open("5gram-it-multi-ds-eos.arpa", "w") as write_file:
    has_added_eos = False
    for line in read_file:
        if not has_added_eos and "ngram 1=" in line:
            count=line.strip().split("=")[-1]
            write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
        elif not has_added_eos and "<s>" in line:
            write_file.write(line)
            write_file.write(line.replace("<s>", "</s>"))
            has_added_eos = True
        else:
            write_file.write(line)

In [8]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [10]:
! cp 5gram-it-multi-ds-eos.arpa gdrive/MyDrive/KenLM/